# Consume messages from an Apache Kafka partitioned topic

In this notebook you will learn how to consume messages to an Apache Kafka topic with two partitions (created with the [dedicated notebook](3-produce-partitioned-topic.ipynb))

![Consume messages from an Apache Kafka Topic](../img/consume.png)

_If you haven't already created and setup your virtual environment, follow the steps from [0-setup.ipynb](0-setup.ipynb)_. You will need the `PizzasPartitioned` topic that you created in `3-producepartintioned-topic.ipynb`.



---

## Create an Apache Kafka Consumer

The next step is to configure the Kafka Consumer. let's recall in the previous notebook. That `Francesco's` message was sent to partition `0` and `Adele's` message was in partition `1`.

```python
producer.produce(
    "pizzaPartitioned",
    key={"id":1},
    value={"id":1, "name":"👨 Francesco", "pizza":"Margherita 🍕"},
    partition=0
)

producer.produce(
    "pizzaPartitioned",
    key={"id":2},
    value={"id":2, "name":"👩 Adele", "pizza":"Hawaii 🍕+🍍+🥓"},
    partition=1
)
```

In `2-consume.ipynb` we saw that all the topics are all loaded. We're going to cre`ate two consumers that belong to the group `pizzaaioli`. `consumer_0` will consume messages in partition `0` and `consumer_1` consumes messages in partition `1`.

In [63]:
import json
import os

from confluent_kafka import DeserializingConsumer, TopicPartition
from dotenv import load_dotenv

load_dotenv()


def json_serializer(msg, s_obj):
    return json.loads(msg.decode('ascii'))

KAFKA_SERVICE_URI = os.getenv("KAFKA_SERVICE_URI")

conf = {
    'bootstrap.servers': KAFKA_SERVICE_URI,
    'client.id': 'myclient',
    'group.id': 'pizzaioli',
    'security.protocol': 'SSL',
    'ssl.ca.location': '../sslcerts/ca.pem',
    'ssl.certificate.location': '../sslcerts/service.cert',
    'ssl.key.location': '../sslcerts/service.key', 
    'value.deserializer': json_serializer,
    'key.deserializer': json_serializer
    }

consumer_0 = DeserializingConsumer(conf)
consumer_0.assign([TopicPartition("pizzaPartitioned", 0)])
consumer_1 = DeserializingConsumer(conf)
consumer_1.assign([TopicPartition("pizzaPartitioned", 1)])

## Consume messages to the `pizzaPartitioned` topic, partition 0

Review and start the code block below. While the block is running. Use the **LAST** block in `3-produce-partitioned-topic.ipynb` to produced more partitioned samples.

In [79]:
from confluent_kafka import TopicPartition, KafkaException

print("Run the Producer in notebook ...")
for consumer in [consumer_0, consumer_1]:
    running = True
    while running:
        if (msg:=consumer.poll(timeout=1.0)) is None:
            continue
        elif msg.error():
                raise KafkaException(msg.error())
        else:
            print(f"Checking Partition {consumer.assignment()[0].partition} \n" + "-"*10)
            print(f"{msg.partition()}:{msg.offset()}: {msg.key()}:{msg.value()}\n\n")
            running = False

Run the Producer in notebook 3
Checking 0 
----------
0:28: {'id': 1}:{'id': 1, 'name': '👨 Francesco', 'pizza': 'Margherita 🍕'}


Checking 1 
----------
1:18: {'id': 2}:{'id': 2, 'name': '👩 Adele', 'pizza': 'Hawaii 🍕+🍍+🥓'}


